# Logistic Growth Models

In the previous chapter we built exponential growth and decline models that were able to model the beginning and end of the pandemic curves. In this chapter, we'll cover the logistic growth model, which has both exponential growth and  decline within the same model.

## Asymptotes

This section provides brief coverage of asymptotes and the types of functions that produce them. An asymptote is the limit of a function's value as its input approaches some number. The exponential models of the previous chapter all had a single asymptote at one end. We need a model that has two asymptotes, one at either end. Let's take a look at a few functions with asymptotes on either end and look for one that fits the shape of our coronavirus case curve.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dashboard.mplstyle')

### Simple asymptotic functions

The simple function $\frac{1}{x}$ has asymptotes on both ends, but is undefined at zero and doesn't match the shape of our coronavirus case curves, so we'll need a different approach.

In [ ]:
x = np.linspace(-20, -0.1, 50)
x1 = np.linspace(0.1, 20, 50)
y = 1 / x
y1 = 1 / x1
fig, ax = plt.subplots()
ax.plot(x, y, color='black')
ax.plot(x1, y1, color='black')
ax.set_title(r'$y = \frac{1}{x}$');

The function $\frac{a}{b + x ^ 2}$ is always defined and always positive. The parameters, $a$ and $b$ control the height and growth rate (steepness of decline from 0).

In [ ]:
x = np.linspace(-5, 5, 100)
y1 = 1 / (1 + x ** 2)
y2 = 4 / (3 + x ** 2)
fig, ax = plt.subplots()
ax.plot(x, y1)
ax.plot(x, y2)
ax.set_title(r'$y = \frac{a}{b + x ^ 2}$');

## S-Curves

While the above curves have asymptotes on either end, they are not suitable for our data. We are looking for a class of functions known as [S-Curves or Sigmoid functions][1] that are shaped like an "S". The most basic form of this function is called the **logistic function** defined by the following equation:

$$f(x) = \frac{1}{1 + e^x}$$

Any base can be used for the exponential, but $e$ is the most common. Let's make a plot of it below.

[1]: https://en.wikipedia.org/wiki/Sigmoid_function

In [ ]:
x = np.linspace(-5, 5, 50)
y = 1 / (1 + np.exp(x))
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title('Simple "S"-Curve - Logistic Function');

This is almost the exact shape we need for our model. We can rotate the curve around the y-axis so that it ends at it's higher asymptote by negating `x`.

In [ ]:
x = np.linspace(-5, 5, 50)
y = 1 / (1 + np.exp(-x))
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title(r'$y = \frac{1}{1 + e ^{-x}}$');

### Controlling the upper asymptote

The numerator controls the scale and determines the upper asymptote. Instead of using the constant value one, we'll model it using variable name `L`.  Here, we use 10,000 as the upper asymptote. The shape of the curve is the exact same, only with the y-values changed.

In [ ]:
L = 10_000
x = np.linspace(-5, 5, 50)
y = L / (1 + np.exp(-x))
plt.plot(x, y)
plt.title(f'Logistic Function with Asymptote at {L:,}');

### Shifting to the right

Notice that when x equals 0, the  curve is at its midpoint (5,000 above). With our data, `x` represents the number of days since the start of the coronavirus pandemic. Since we can't have negative x-values, we can fix this by introducing a new term, $x_0$, that shifts the graph horizontally. The value for $x_0$ will be the location of the midpoint, that is, $\frac{L}{2}$.

$$f(x) = \frac{L}{1 + e^{-(x - x_0)}}$$

In [ ]:
L = 10_000
x0 = 50
x = np.linspace(40, 60)
y = L / (1 + np.exp(-(x - x0)))
plt.plot(x, y)
plt.title('Logistic Function - Scaled and Shifted');

### Control steepness with growth rate

The growth rate (steepness of the curve) can be modified by adding another parameter, $k$, to the equation. The above curves had a growth rate of $k = 1$. Here, we change it to a much smaller number, $0.1$, which significantly flattens the curve. Notice the x-axis ranges from 0 to 100 now instead of from 40 to 60.

$$f(x) = \frac{L}{1 + e^{-k(x - x_0)}}$$

In [ ]:
L = 10_000
x0 = 50
k = 0.1
x = np.linspace(0, 100, 50)
y = L / (1 + np.exp(-k * (x - x0)))
plt.plot(x, y)
plt.title('Logistic Function - Scaled, Shfited, and Flattened');

### Exercise 23

<span style="color:green; font-size:16px">Write a function that computes the logistic function given values of x and its three parameters.</span>

In [ ]:
def logistic_func(x, L, x0, k):
    """
    Computes the value of the logistic function
    
    Parameters
    ----------
    x : array of x values
    
    L : Upper asymptote
    
    x0 : horizontal shift
    
    k : growth rate
    """
    pass

In [ ]:
from solutions import logistic_func
logistic_func(np.arange(20), 5000, 12, 0.1).round(-1)

### Exercise 24

<span style="color:green; font-size:16px">Execute the code below, which reads in the data, selects cases from Italy from February 15th through June 1st, and plots the curve. Then write a function that accepts a Series of data and the three parameters for the logistic function. Have the function plot the given data against the logistic function. Experiment with different parameter values until you produce a curve that closely resembles the one from Italy.</span>

In [ ]:
from prepare import PrepareData
data = PrepareData(download_new=False).run()
italyc = data['world_cases']['Italy']
italyc = italyc.loc["2020-02-15":"2020-06-01"]
italyc.plot(title="Italy - Cumulative Cases");

In [ ]:
def logistic_guess_plot(s, L, x0, k):
    """
    Plots the given series of data along with its 
    estimated values using the logistic function.
    
    Parameters
    ----------
    s : Series of actual data
    
    L, x0, k : same as above
    
    Returns
    -------
    None
    """
    pass

# After completing this function, call it multiple times 
# with different values of L, x0, and k until it closely
# matches the actual curve from Italy.

## Estimating logistic function parameters

Now that we have the function that follows the shape of our data, we can use scipy's `least_squares` again to automatically find the parameters that fit the data the best. Before doing so, let's smooth our data. We'll only let our model see data through April 1.

In [ ]:
from solutions import smooth
last_date = '2020-04-01'
y = italyc.loc[:last_date]
y_smooth = smooth(y, n=15)
y_smooth.head()

In [ ]:
y_smooth.tail()

The smoothed data is plotted alongside the actual data below.

In [ ]:
y.plot(label='actual', title="Italy Cases - Training");
y_smooth.plot(label='smoothed').legend();

### Parameter bounds

The parameter bounds are important to get correct as the trajectory of the curve can vary widely with just small deviations. The parameter `L` represents the upper asymptote, or the maximum number of cases. Given the current state of the data on April 1, it's difficult to place a reasonable bound for it. We can choose the last known value as the minimum, and a very large number as the maximum.

The parameter `x0` represents the midpoint (also known as the inflection point) and is the number of days since the start where half of the cases have occurred.

The parameter `k` represents the growth rate, or the steepness of the curve. A larger `k` creates a steeper curve, and a smaller `k` creates a flatter curve. It can be helpful to create multiple curves for different values of `k` while keeping `L` and `x0` constant to better understand the logistic function.

### Exercise 25

<span style="color:green; font-size:16px">Write a function that accepts the data to be modeled as a Series, a list of potential values of `k`, a single value for `L`, and a single value for `x0`. Have it plot the logistic curves starting on the same date as the passed data and continuing for twice the amount of `x0`.</span>

In [ ]:
def plot_ks(s, ks, L, x0):
    """
    Plots the data to be modeled along with the logistic curves
    for each of the given ks on the same plot. This function
    helps find good bounds for k in curve_fit.
    
    Parameters
    ----------
    s : data to be modeled
    
    ks : list of floats for different values of the growth rate
    
    L : Upper asymptote
    
    x0 : horizontal shift
    
    Returns
    -------
    None
    """
    pass

We'll make a few different calls to `plot_ks` pairing up very high and very low values of `L` and `x0` to see if we can triangulate the area where `k` might be. Below, we have a scenario where `L` is 1,000,000 and $x_0$ is 100 days. Comparing the known, smoothed data to the steepness of the other curves would make `k` less than 0.1 and perhaps greater than 0.01.

In [ ]:
from solutions import plot_ks
ks = [0.03, 0.1, 0.3, 0.5]
plot_ks(y_smooth, ks, 1_000_000, 100)

When using a shorter amount of time to the mid-point and an `L` of 200,000, the shape of the smoothed curve is closer to 0.1. 

In [ ]:
ks = [0.03, 0.1, 0.3, 0.5]
plot_ks(y_smooth, ks, 200_000, 50)

If we assume just a small future increase in total cases, the smoothed slope is closer to 0.3.

In [ ]:
ks = [.05, 0.1, 0.3, 0.5]
plot_ks(y_smooth, ks, 150_000, 50)

From this analysis, it appears that `k` will almost certainly be less than 0.5. By definition, it must be positive (or else the curve would flip). Let's use a lower bound of 0.01. We set the upper bounds of `L` and `x0` to 1,000,000 and 150 and use the lower bound as the initial guess.

In [ ]:
lower = y_smooth[-1], 20, 0.01
upper = 1_000_000, 150, 0.5
bounds = lower, upper
p0 = lower

We use `least_squares` just as before to get the fitted parameter values.

In [ ]:
from solutions import logistic_func, optimize_func
from scipy.optimize import least_squares
y = y_smooth
x = np.arange(len(y))
res = least_squares(optimize_func, p0, args=(x, y, logistic_func), bounds=bounds)
L, x0, k = res.x
print(f'L = {L:,.0f}\nx0 = {x0:.0f}\nk = {k:.3f}')

Let's use our handy function `predict_all` to plot the next 50 days with this model.

In [ ]:
from solutions import predict_all
predict_all(italyc, start_date=None, last_date="2020-04-01", n_pred=50, n_smooth=15, 
            model=logistic_func, bounds=bounds, p0=p0, title="Italy");

Although we have the right shape for the curve, this prediction quickly fell off the target. Let's test out our model with more training data by allowing it to see data up to May 1. Unfortunately, the result is still poor.

In [ ]:
predict_all(italyc, start_date=None, last_date="2020-05-01", n_pred=30, n_smooth=15, 
            model=logistic_func, bounds=bounds, p0=p0, title="Italy");

## Generalized Logistic Function

Our first results using the logistic model do not look good as the curve is flattening much too fast. The basic form of the logistic function does not allow for asymmetry. The current form of the logistic function is perfectly symmetric - the first half of the curve is an exact mirror of the second half. 

The overwhelming majority of areas have cases/deaths that slowly trail off towards a maximum. The current form of the logistic function cannot fit this kind of data. There exists a more broad [generalized logistic function][0] that is able to be tuned such that a long tail is possible. One form is written below.

$$f(x) = \frac{L}{(1 + e^{-k(x - x_0)})^{\frac{1}{v}}}$$

`L`, and `k` represent the same values as before, the maximum of the curve and the growth rate. The new parameter `v` changes the symmetry of the curve and shifts the entire curve horizontally. The following summarizes `v`, which must be positive:

* `v < 1`
    * growth is faster before the midpoint
    * curve shifted to the right
* `v > 1`
    * growth is faster after the midpoint
    * curve shifted to the left


Let's create a function that computes the value for the generalized logistic function.

[0]: https://en.wikipedia.org/wiki/Generalised_logistic_function

In [ ]:
def general_logistic(x, L, x0, k, v):
    return L / ((1 + np.exp(-k * (x - x0))) ** (1 / v))

Here, we choose a set of parameters that show our new S-curve. Take note that although `x0` is 0. The midpoint (`y=500`) occurs at about `x=35`.

In [ ]:
L, x0, k, v = 1_000, 0, 0.05, 0.1
x = np.arange(120)
y = general_logistic(x, L, x0, k, v)
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_title(f"Generalized Logistic Function with L={L:,}, x0={x0}, k={k}, v={v}");

When `v` equals 1, the equation simplifies to our previous logistic function which has its midpoint at `x0=0`. The same generalized logistic function with `v` set to 0.1 is plotted below along with a simple logistic function with `v` equal to 1. Notice how `v` shifts the generalized logistic function much further to the right even though they have the same values for `L`, `x0`, and `k`.

In [ ]:
# same as above
L, x0, k, v = 1_000, 0, 0.05, 0.1
x = np.arange(120)
y = general_logistic(x, L, x0, k, v)

# v set to 1 - simple logistic function
L, x0, k, v = 1_000, 0, 0.05, 1
x1 = np.arange(-60, 60)
y1 = general_logistic(x1, L, x0, k, v)

fig, ax = plt.subplots()
ax.plot(x, y, label='Generalized Logistic Function with v=0.1')
ax.plot(x1, y1, label='Simple Logistic Function with v=1')
ax.legend(bbox_to_anchor=(0, -.1), loc='upper left')
ax.set_title(f"Generalized vs Simple Logistic Function with L={L:,}, x0={x0}, k={k}");

### Plotting Asymmetry

To get a better understanding of the asymmetry created by the `v` parameter, the following function is defiend which plots the generalized logistic function along with horizontal lines at the 10th, 50th, and 90th percent of `L`. It then calculates and displays the number of days it takes to go from the 10th to 50th percent and from the 50th to 90th percent. When `v` is less than 1, it will take less days to from the 10th to 50th percent than it does from the 50th to 90th percent. The opposite is true when `v` is greater than 1.

In [ ]:
def plot_asymmetry(x, L, x0, k, v):
    y = general_logistic(x, L, x0, k, v)
    fig, ax = plt.subplots()
    ax.plot(x, y)
    low, mid, high = int(0.1 * L), int(0.5 * L), int(0.9 * L)
    ax.hlines([low, mid, high], x[0], x[-1], color='black', ls='--')
    
    days_to_10 = np.argmax(y > low)
    days_to_50 = np.argmax(y > mid)
    days_to_90 = np.argmax(y > high)
    days_10_to_50 = days_to_50 - days_to_10
    days_50_to_90 = days_to_90 - days_to_50
    ax.set_title(f"Generalized Logistic Function with L={L:,}, k={k}, v={v}")
    ax.text(0.05, 0.75, f'Days from {low} to {mid}: {days_10_to_50}', transform=ax.transAxes, size=8)
    ax.text(0.05, 0.68, f'Days from {mid} to {high}: {days_50_to_90}', transform=ax.transAxes, size=8)

We will hold `L`, `x0`, and `k` constant and change `v` to show the asymmetry. Here, when `v` is 0.1 it takes 13 less days to go from 100 to 500, than it does from 500 to 900. This value of `v` would be used whenever a curve has a longer tail like the coronavirus case curve for most areas.

In [ ]:
x = np.arange(120)
plot_asymmetry(x, L=1000, x0=0, k=0.05, v=0.1)

Here, we set `v` to 1 simplifying the logistic function so that it will be perfectly symmetric again. The one day difference observed below is due to rounding.

In [ ]:
x = np.arange(-60, 60)
plot_asymmetry(x, L=1000, x0=0, k=0.05, v=1)

Here, we set the value of `v` to be 2, which has the effect of the curve reaching its right assymptote faster.

In [ ]:
x = np.arange(-120, 60)
plot_asymmetry(x, L=1000, x0=0, k=0.05, v=2)

The total time from 10th to 90th percent was nearly double (121 vs 63) when `v` was 2 as opposed to 0.1 in the first graph. The growth rate, `k`, is affected by the value of `v`. To decrease the total amount of time from the 10th to 90th percent, we increase `k` as `v` increases. Here, we set `k` to 0.1.

In [ ]:
x = np.arange(-60, 40)
plot_asymmetry(x, L=1000, x0=0, k=0.1, v=2)

### What does `x0` represent now?

The parameter `x0` still shifts the data horizontally, but no longer represents the midpoint of the graph. Here, we plot different values of `x0` holding the other parameters constant. The curve is the exact same shape, just at a different location on the x-axis.

In [ ]:
L, k, v = 1_000, 0.05, 0.1
x = np.arange(-20, 150)

fig, ax = plt.subplots()
for x0 in [-20, 0, 20]:
    y = general_logistic(x, L, x0, k, v)
    ax.plot(x, y, label=f'x0={x0}')
ax.legend()
ax.set_title(f"Generalized vs Simple Logistic Function with L={L:,}, k={k}, v={v}");

## Predictions with the generalized logistic function

With our `predict_all` function, it's easy to plug in a new model and see the results. We just need to change the limits and set the bounds of the parameters. We use the same limits as before for `L` and `k`. We allow `x0` to be positive or negative as `v` also shifts the data horizontally. For `v`, some experimentation was needed to find the feasible limits.

In [ ]:
last_date = "2020-04-01"
L_min, L_max = italyc[last_date], 1_000_000
x0_min, x0_max = -50, 50
k_min, k_max = 0.01, 0.5
v_min, v_max = 0.01, 2
lower = L_min, x0_min, k_min, v_min
upper = L_max, x0_max, k_max, v_max
bounds = lower, upper
p0 = L_max, 0, 0.1, 0.1

params, y_pred = predict_all(italyc, start_date=None, last_date="2020-05-01", n_smooth=15,
                             n_pred=30, model=general_logistic, bounds=bounds, p0=p0, 
                             title="Italy - Generalized Logistic Function");

The new model provides an excellent fit and predicts future cases almost perfectly. Let's view the optimal parameters.

In [ ]:
params

### Have we built the perfect model?

Although our model provides a very good fit, we won't be able to reproduce this performance on all other countries or other time periods. For instance, our model performs worse if we only allow it to see data only up through March 15.

In [ ]:
predict_all(italyc, start_date=None, last_date="2020-03-15", n_smooth=15, 
            n_pred=50, model=general_logistic, bounds=bounds, p0=p0, 
            title="Italy - Generalized Logistic Function");

## Modeling other countries

Let's model the cases from a few more countries, defining a function to automate the process for us. It chooses the initial guess and upper bound for `L` to be 5 and 1,000 times the last value.

In [ ]:
def model_country(data, name, start_date, last_date):
    s = data['world_cases'][name]    
    L_min, L_max = s.iloc[0], s.iloc[-1] * 1000
    x0_min, x0_max = -50, 50
    k_min, k_max = 0.01, 0.5
    v_min, v_max = 0.01, 2
    lower = L_min, x0_min, k_min, v_min
    upper = L_max, x0_max, k_max, v_max
    bounds = lower, upper
    p0 = L_min * 5, 0, 0.1, 0.1
    predict_all(s, start_date=start_date, last_date=last_date, n_smooth=15, 
                n_pred=50, model=general_logistic, bounds=bounds, p0=p0, 
                title=f"{name} - Generalized Logistic Function");

In [ ]:
model_country(data, "United Kingdom", "2020-03-01", "2020-03-30")

In [ ]:
model_country(data, "Belgium", "2020-03-01", "2020-04-10")

In [ ]:
model_country(data, "Germany","2020-03-01", "2020-03-30")

In [ ]:
model_country(data, "South Korea","2020-02-15", "2020-03-1")

While a single generalized logistic function works fairly well for modeling a single wave with exponential growth and decline, it will need to be modified in order to handle new waves of the virus. We'll see an approach for modeling this in the next chapter.